In [231]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [51]:
data = pd.read_csv('./data/Final_2.csv')

In [52]:
data.drop(columns=['Unnamed: 0'],inplace=True)

In [53]:
data.head()

,raceId,driverId,lt_lap,position,milliseconds,circuitId,date,ps_lap,pit_time,target
0,841,153,1,21,133879,1,2011-03-27,1.0,26898.0,True
1,841,30,1,22,161428,1,2011-03-27,1.0,25021.0,True
2,841,17,11,7,114995,1,2011-03-27,11.0,23426.0,True
3,841,4,12,12,112624,1,2011-03-27,12.0,23251.0,True
4,841,13,13,11,114060,1,2011-03-27,13.0,23842.0,True


In [79]:
num =data.head()

In [117]:
new=[data[(data.raceId == data.loc[i].raceId) & 
     (data.driverId == data.loc[i].driverId) & 
     (data.lt_lap == (data.loc[i].lt_lap-1))].milliseconds.values[0]+1000
      if ((data.loc[i].target) and (data.loc[i].lt_lap != 1)) else data.loc[i].milliseconds for i in range(len(data))]

In [118]:
data['up_milli'] = new

In [129]:
data.date = [x[:4] for x in data.date]

In [237]:
data.to_csv("./data/For_models.csv")

In [238]:
data = pd.read_csv("./data/For_models.csv")

In [200]:
features = ['lt_lap', 'position', 'milliseconds', 'circuitId', 'date']

In [222]:
X= data[features1]
y= data['target']

In [225]:
y.value_counts(normalize=True)

False    0.984263
True     0.015737
Name: target, dtype: float64

In [223]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    random_state=42)

In [13]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
logreg.score(X_test, y_test)

0.984120774464974

In [205]:
features1 = ['lt_lap', 'position', 'up_milli', 'circuitId', 'date']
X= data[features1]
y= data['target']

In [206]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    random_state=42)

In [136]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [137]:
logreg.score(X_test, y_test)

0.9841400142374316

R**2

In [215]:
pred = logreg.predict(X_test)

In [216]:
senistivity_specificity(y_test,pred)

Sensitivity: 0.0
Specificity: 1.0


In [217]:
dt1 = DecisionTreeClassifier(random_state = 42)

In [218]:
dt1.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [219]:
dt1.score(X_test, y_test)

0.978021766595907

In [220]:
out = dt1.predict(X_test)

In [221]:
senistivity_specificity(y_test,out)

Sensitivity: 0.3085
Specificity: 0.9888


In [207]:
grid = GridSearchCV(estimator = DecisionTreeClassifier(),
                    param_grid = {'max_depth': [10,15,20,25],
                                  'min_samples_split': [20,25,30,35],
                                  'min_samples_leaf': [2, 3, 4, 5, 6, 7]},
                    cv = 5,
                    verbose = 1)

In [208]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed:  4.9min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [10, 15, 20, 25],
                      

In [209]:
grid.best_score_

0.9846261728549388

In [210]:
dt = grid.best_estimator_

In [195]:
def senistivity_specificity(y_t, predI):
    tn, fp, fn, tp = confusion_matrix(y_t,
                                  predI).ravel()
    sens = tp / (tp + fn)
    print(f'Sensitivity: {round(sens, 4)}')
    spec = tn / (tn + fp)
    print(f'Specificity: {round(spec, 4)}')

In [213]:
preds = dt.predict(X_test)

In [214]:
senistivity_specificity(y_test, preds)

Sensitivity: 0.038
Specificity: 0.9997


In [227]:
rf = RandomForestClassifier()

In [228]:
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [229]:
preds = rf.predict(X_test)

In [230]:
senistivity_specificity(y_test,preds)

Sensitivity: 0.1084
Specificity: 0.9995


In [232]:
et = ExtraTreesClassifier()

In [233]:
et.fit(X_train,y_train)
preds=rf.predict(X_test)

In [234]:
senistivity_specificity(y_test,preds)

Sensitivity: 0.1084
Specificity: 0.9995
